# Football

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from time import sleep
import json
import re

In [2]:
def get_pages_season(link):

    global driver

    driver = webdriver.Chrome()

    driver.get(link)

    driver.refresh()

    sleep(10)

    pages = driver.find_elements(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/main/div[3]/div[4]/div[1]/div[3]/div/a')

    pages_list = [elem.text for elem in pages]

    filtered_list = [
        item for item in pages_list
        if item != 'Next' 
    ]

    driver.close()

    return filtered_list

In [3]:
def get_match_odds(link):

    driver.get(link)

    driver.refresh()

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    sleep(5)

    bookmarkers = driver.find_elements(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/main/div[3]/div[2]/div[2]/div/div/div[1]')

    bookmarkers_table = [row.text.split("\n") for row in bookmarkers]

    filtered_data = [
    item for item in bookmarkers_table[0] 
    if item != 'BONUS' and not item.startswith('CLICK TO SHOW') 
    ]

    # Dividir la lista en grupos de 5 elementos
    rows = [filtered_data[i:i + 5] for i in range(0, len(filtered_data), 5)]

    # Crear el DataFrame
    df = pd.DataFrame(rows[1:], columns=rows[0])  # Ignoramos la primera fila ya que es el header

    df_2 = pd.melt(df, id_vars='Bookmakers', value_vars=['1', 'X', '2','Payout'])

    df_2["Var"] = df_2['Bookmakers'] +"_"+ df_2["variable"]

    result = df_2[["Var", "value"]].transpose()

    result.columns = result.iloc[0]
    result = result[1:]

    result.reset_index(drop=True, inplace=True)

    result = result.reindex(sorted(result.columns), axis=1)

    return result

In [4]:
def get_season_data(link,page):

    global driver

    driver = webdriver.Chrome()
    
    url = '{}#/page/{}'.format(link,page)

    driver.get(url)

    driver.refresh()

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    sleep(5)

    gameRows = driver.find_elements(By.XPATH,'//div[@class="group flex"]')

    rowData = [row.text.split("\n") for row in gameRows]

    columns = ['hora', 'h_team', 'h_score', 'marker', 'a_score', 'a_team', 'notes', 'OP_1', 'OP_X', 'OP_2', 'n_bookmkrs']

    filtered_data = [
    item for item in rowData
    if item != 'Add to my coupon' 
    ]

    complete_data = [
        row[:2] + [None] + row[2:3] + [None] + row[3:] if len(row) == 9 else 
        row
        for row in filtered_data
    ]

    normalized_data = [
        row[:6] + [None] + row[6:] if len(row) == 10 else row  
        for row in complete_data
    ]

    df_event = pd.DataFrame(normalized_data, columns=columns)

    df_event = df_event.fillna(value=np.nan)

    #df_event = pd.DataFrame(rowData, columns=['hora', 'h_team', 'h_score', 'marker', 'a_score', 'a_team', 'OP_1', 'OP_X', 'OP_2', 'n_bookmkrs'])

    matches = driver.find_elements(By.XPATH,'//div[@class="group flex"]/a')

    matches_links = [elem.get_attribute('href') for elem in matches]

    matches_odds = []

    for x in matches_links:

        df_match_odds = get_match_odds(link=x)
        
        matches_odds.append(df_match_odds)

    
    df_extra_odds = pd.concat(matches_odds)

    df_event.reset_index(inplace=True, drop=True)

    df_extra_odds.reset_index(inplace=True, drop=True)

    df_all = pd.concat([df_event, df_extra_odds], axis=1)

    driver.close()

    return df_all


In [5]:
# Función para extraer números de una lista de URLs
def extract_numbers(links):
    all_numbers = []
    for link in links:
        numbers = re.findall(r'\d+', link)
        all_numbers.extend(numbers)
    return all_numbers

In [6]:
season_links = [
    "https://www.oddsportal.com/football/colombia/liga-postobon-2014/results/",
    "https://www.oddsportal.com/football/colombia/liga-aguila-2015/results/",
    "https://www.oddsportal.com/football/colombia/liga-aguila-2016/results/",
    "https://www.oddsportal.com/football/colombia/liga-aguila-2017/results/",
    "https://www.oddsportal.com/football/colombia/liga-aguila-2018/results/",
    "https://www.oddsportal.com/football/colombia/liga-aguila-2019/results/",
    "https://www.oddsportal.com/football/colombia/primera-a-2020/results/",
    "https://www.oddsportal.com/football/colombia/primera-a-2021/results/",
    "https://www.oddsportal.com/football/colombia/primera-a-2022/results/",
    "https://www.oddsportal.com/football/colombia/primera-a-2023/results/"
]


numbers = extract_numbers(season_links)

print(numbers)


['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [8]:
list_season_data = []

for idx, x in enumerate(season_links):

    print(numbers[idx])

    pages_season = get_pages_season(link=x)

    list_results = []

    for y in pages_season:

        df_current = get_season_data(link=x,page=y)

        sleep(20)
        
        list_results.append(df_current)
    
    df_consolidado = pd.concat(list_results)

    df_consolidado['Temporada'] = numbers[idx]

    list_season_data.append(df_consolidado)

    print("Done!")

2014


C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-

Done!
2015


C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-

Done!
2016


...

Done!
2022


C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-

Done!
2023


C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event = df_event.fillna(value=np.nan)
C:\Users\Victor\AppData\Local\Temp\ipykernel_19228\1249643821.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-

Done!


In [9]:
df_all_seasons = pd.concat(list_season_data)

In [10]:
df_all_seasons

,hora,h_team,h_score,marker,a_score,a_team,notes,OP_1,OP_X,OP_2,...,Lasbet_Payout,Lasbet_X,Alphabet_1,Alphabet_2,Alphabet_Payout,Alphabet_X,VOBET_1,VOBET_2,VOBET_Payout,VOBET_X
0,17:30,Santa Fe,1,–,1,Ind. Medellin,NaN,1.85,3.51,4.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19:45,Uniautonoma,2,–,0,Quindio,NaN,1.75,3.32,4.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19:30,Ind. Medellin,1,–,2,Santa Fe,NaN,1.96,3.33,3.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15:30,Quindio,0,–,0,Uniautonoma,NaN,2.11,3.24,3.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18:15,Atl. Nacional,0,–,1,Santa Fe,NaN,1.52,3.83,6.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,18:10,Deportes Tolima,2,–,1,America De Cali,NaN,2.05,3.05,4.07,...,96.0%,3.10,1.99,3.90,90.9%,2.93,1.99,3.90,90.9%,2.93
48,16:00,Chico,2,–,0,Alianza,NaN,1.85,3.09,5.06,...,95.6%,3.07,1.80,5.00,90.9%,2.90,1.80,5.00,90.9%,2.90
49,17:20,Aguilas,1,–,1,Junior,NaN,2.43,2.93,3.23,...,95.7%,2.93,2.34,3.20,90.9%,2.78,2.40,3.12,91.0%,2.76
0,15:10,U. Magdalena,2,–,1,Huila,NaN,1.88,3.21,4.54,...,95.7%,3.21,1.81,4.60,90.9%,3.03,1.81,4.60,90.9%,3.03


In [11]:
df_all_seasons.to_csv('historical_odds_colombia_primera_a.csv', sep=';', encoding='utf-8', index=False, header=True)

# BORRADOR

In [24]:
x,y

('https://www.oddsportal.com/football/colombia/liga-aguila-2015/results/', '1')

In [ ]:
df_current = get_season_data(link=x,page=y)

In [143]:
df_current = get_season_data(link='https://www.oddsportal.com/football/colombia/primera-a-2020/results/',page=1)

In [144]:
df_current

,hora,h_team,h_score,marker,a_score,a_team,notes,OP_1,OP_X,OP_2,...,William Hill_Payout,William Hill_X,bet-at-home_1,bet-at-home_2,bet-at-home_Payout,bet-at-home_X,bet365_1,bet365_2,bet365_Payout,bet365_X
0,18:30,Dep. Cali,2,–,1,Millonarios,pen.,1.62,3.60,5.58,...,93.2%,3.40,1.64,5.00,91.6%,3.55,1.61,5.00,91.0%,3.60
1,18:00,Santa Fe,2,–,0,America De Cali,NaN,1.73,3.36,5.32,...,93.1%,3.30,1.67,5.20,91.4%,3.30,1.70,6.00,93.7%,3.20
2,20:00,Millonarios,1,–,0,Pereira,NaN,1.51,3.91,6.62,...,93.5%,3.80,1.52,6.20,91.8%,3.70,1.50,7.00,93.2%,3.80
3,18:00,America De Cali,3,–,0,Santa Fe,NaN,2.57,2.77,3.22,...,93.6%,2.75,2.45,3.15,91.8%,2.75,2.60,3.10,94.0%,2.80
4,19:40,Millonarios,1,–,0,Once Caldas,pen.,1.63,3.55,5.81,...,93.7%,3.60,1.60,5.40,91.6%,3.55,1.66,6.00,93.3%,3.30
5,19:40,Pereira,2,–,1,Bucaramanga,pen.,2.44,2.77,3.35,...,93.5%,2.80,2.38,3.30,92.0%,2.75,2.30,3.60,92.3%,2.70
6,20:00,Junior,1,–,2,America De Cali,NaN,2.29,2.87,3.47,...,93.5%,2.80,2.29,3.40,91.9%,2.80,2.20,3.25,91.3%,3.00
7,19:30,Santa Fe,2,–,1,La Equidad,NaN,1.71,3.26,5.49,...,93.1%,3.20,1.72,5.40,91.8%,3.10,1.61,5.50,91.2%,3.40
8,19:40,Aguilas,2,–,1,Ind. Medellin,NaN,2.04,3.29,3.70,...,93.7%,3.25,1.99,3.50,91.6%,3.30,1.95,4.20,94.0%,3.20
9,19:40,Alianza,2,–,2,Jaguares de Cordoba,NaN,2.33,3.04,3.24,...,94.0%,3.00,2.26,3.20,91.9%,3.00,2.30,3.40,94.1%,3.00


In [66]:
x,y

('https://www.oddsportal.com/football/colombia/primera-a-2023/results/', '10')

In [ ]:
gameRows = driver.find_elements(By.XPATH,'//div[@class="group flex"]')

rowData = [row.text.split("\n") for row in gameRows]

In [119]:
driver = webdriver.Chrome()

#page = filtered_list[0]

url = '{}#/page/{}'.format('https://www.oddsportal.com/football/colombia/primera-a-2020/results/',1)

driver.get(url)

driver.refresh()

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


sleep(5)

#gameRows = driver.find_elements(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/main/div[3]/div[4]/div[1]/div[1]/div')
# //*[@id="bqfTHpLu"]/div/div
# /html/body/div[1]/div[1]/div[1]/div/main/div[3]/div[4]/div[1]/div[1]/div[31]/div/div
#gameRows = driver.find_elements(By.XPATH,'//div[@class="group flex"]/div')
gameRows = driver.find_elements(By.XPATH,'//div[@class="group flex"]')

rowData = [row.text.split("\n") for row in gameRows]

matches = driver.find_elements(By.XPATH,'//div[@class="group flex"]/a')

matches_links = [elem.get_attribute('href') for elem in matches]

In [120]:
rowData

[['18:30',
  'Dep. Cali',
  '2',
  '–',
  '1',
  'Millonarios',
  'pen.',
  '1.62',
  '3.60',
  '5.58',
  '8'],
 ['18:00',
  'Santa Fe',
  '2',
  '–',
  '0',
  'America De Cali',
  '1.73',
  '3.36',
  '5.32',
  '8'],
 ['20:00',
  'Millonarios',
  '1',
  '–',
  '0',
  'Pereira',
  '1.51',
  '3.91',
  '6.62',
  '8'],
 ['18:00',
  'America De Cali',
  '3',
  '–',
  '0',
  'Santa Fe',
  '2.57',
  '2.77',
  '3.22',
  '8'],
 ['19:40',
  'Millonarios',
  '1',
  '–',
  '0',
  'Once Caldas',
  'pen.',
  '1.63',
  '3.55',
  '5.81',
  '8'],
 ['19:40',
  'Pereira',
  '2',
  '–',
  '1',
  'Bucaramanga',
  'pen.',
  '2.44',
  '2.77',
  '3.35',
  '7'],
 ['20:00',
  'Junior',
  '1',
  '–',
  '2',
  'America De Cali',
  '2.29',
  '2.87',
  '3.47',
  '7'],
 ['19:30',
  'Santa Fe',
  '2',
  '–',
  '1',
  'La Equidad',
  '1.71',
  '3.26',
  '5.49',
  '7'],
 ['19:40',
  'Aguilas',
  '2',
  '–',
  '1',
  'Ind. Medellin',
  '2.04',
  '3.29',
  '3.70',
  '7'],
 ['19:40',
  'Alianza',
  '2',
  '–',
  '2',
  'J

In [121]:
matches_links

['https://www.oddsportal.com/football/colombia/primera-a-2020/dep-cali-millonarios-t0uJrkFt/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/santa-fe-america-de-cali-A3ZL5k3b/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/millonarios-dep-pereira-WSxjpAlj/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/america-de-cali-santa-fe-d0VH6Vmh/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/millonarios-once-caldas-vPWc9ZPO/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/dep-pereira-bucaramanga-0YXgAFAI/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/junior-america-de-cali-YslCO8X4/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/santa-fe-la-equidad-QHp4QU2h/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/aguilas-doradas-ind-medellin-lM6srSQp/',
 'https://www.oddsportal.com/football/colombia/primera-a-2020/alianza-jaguares-de-cordoba-UHqU5VIi/',
 'https://www.odd

In [117]:
[row.text for row in gameRows]

['18:30\nDep. Cali\n2\n–\n1\nMillonarios\npen.\n1.62\n3.60\n5.58\n8',
 '18:00\nSanta Fe\n2\n–\n0\nAmerica De Cali\n1.73\n3.36\n5.32\n8',
 '20:00\nMillonarios\n1\n–\n0\nPereira\n1.51\n3.91\n6.62\n8',
 '18:00\nAmerica De Cali\n3\n–\n0\nSanta Fe\n2.57\n2.77\n3.22\n8',
 '19:40\nMillonarios\n1\n–\n0\nOnce Caldas\npen.\n1.63\n3.55\n5.81\n8',
 '19:40\nPereira\n2\n–\n1\nBucaramanga\npen.\n2.44\n2.77\n3.35\n7',
 '20:00\nJunior\n1\n–\n2\nAmerica De Cali\n2.29\n2.87\n3.47\n7',
 '19:30\nSanta Fe\n2\n–\n1\nLa Equidad\n1.71\n3.26\n5.49\n7',
 '19:40\nAguilas\n2\n–\n1\nInd. Medellin\n2.04\n3.29\n3.70\n7',
 '19:40\nAlianza\n2\n–\n2\nJaguares de Cordoba\n2.33\n3.04\n3.24\n7',
 '19:40\nEnvigado\n0\n–\n1\nPereira\n3.03\n3.01\n2.47\n7',
 '19:40\nMillonarios\n2\n–\n0\nChico\n1.26\n5.12\n12.63\n7',
 '19:40\nOnce Caldas\n2\n–\n0\nPatriotas\n1.70\n3.69\n4.80\n7',
 '20:00\nAmerica De Cali\n0\n–\n0\nJunior\n2.70\n2.89\n2.88\n7',
 '20:00\nLa Equidad\n1\n–\n1\nSanta Fe\n3.60\n2.95\n2.20\n7',
 '15:00\nOnce Caldas\n

In [107]:
rowData

[['1.62'],
 ['3.60'],
 ['5.58'],
 ['8'],
 ['1.73'],
 ['3.36'],
 ['5.32'],
 ['8'],
 ['1.51'],
 ['3.91'],
 ['6.62'],
 ['8'],
 ['2.57'],
 ['2.77'],
 ['3.22'],
 ['8'],
 ['1.63'],
 ['3.55'],
 ['5.81'],
 ['8'],
 ['2.44'],
 ['2.77'],
 ['3.35'],
 ['7'],
 ['2.29'],
 ['2.87'],
 ['3.47'],
 ['7'],
 ['1.71'],
 ['3.26'],
 ['5.49'],
 ['7'],
 ['2.04'],
 ['3.29'],
 ['3.70'],
 ['7'],
 ['2.33'],
 ['3.04'],
 ['3.24'],
 ['7'],
 ['3.03'],
 ['3.01'],
 ['2.47'],
 ['7'],
 ['1.26'],
 ['5.12'],
 ['12.63'],
 ['7'],
 ['1.70'],
 ['3.69'],
 ['4.80'],
 ['7'],
 ['2.70'],
 ['2.89'],
 ['2.88'],
 ['7'],
 ['3.60'],
 ['2.95'],
 ['2.20'],
 ['7'],
 ['1.58'],
 ['3.70'],
 ['6.13'],
 ['7'],
 ['5.40'],
 ['3.61'],
 ['1.64'],
 ['7'],
 ['2.37'],
 ['3.08'],
 ['3.12'],
 ['7'],
 ['3.28'],
 ['2.91'],
 ['2.39'],
 ['7'],
 ['2.71'],
 ['2.82'],
 ['2.88'],
 ['7'],
 ['3.28'],
 ['2.90'],
 ['2.40'],
 ['7'],
 ['2.58'],
 ['2.87'],
 ['3.02'],
 ['7'],
 ['1.68'],
 ['3.38'],
 ['5.58'],
 ['7'],
 ['2.65'],
 ['3.22'],
 ['2.59'],
 ['7'],
 ['1.55'],
 ['3

In [80]:
[row.text.split("\n") for row in gameRows]

[['20:00',
  'Ind. Medellin',
  '2',
  '–',
  '2',
  'Junior',
  'pen.',
  '1.72',
  '3.55',
  '4.95',
  '13'],
 ['16:00',
  'Junior',
  '3',
  '–',
  '2',
  'Ind. Medellin',
  '1.96',
  '3.19',
  '4.18',
  '13'],
 ['20:15',
  'Ind. Medellin',
  '2',
  '–',
  '1',
  'America De Cali',
  '2.25',
  '3.31',
  '3.17',
  '13'],
 ['20:15',
  'Junior',
  '4',
  '–',
  '2',
  'Deportes Tolima',
  '1.90',
  '3.64',
  '3.82',
  '13'],
 ['18:00',
  'Aguilas',
  '3',
  '–',
  '1',
  'Dep. Cali',
  '1.38',
  '4.70',
  '8.21',
  '13'],
 ['18:00',
  'Millonarios',
  '0',
  '–',
  '1',
  'Atl. Nacional',
  '1.83',
  '3.41',
  '4.45',
  '13'],
 ['19:00',
  'America De Cali',
  '1',
  '–',
  '0',
  'Millonarios',
  '2.39',
  '3.18',
  '3.05',
  '13'],
 ['15:30',
  'Atl. Nacional',
  '0',
  '–',
  '5',
  'Ind. Medellin',
  '2.93',
  '3.22',
  '2.45',
  '13'],
 ['20:15', 'Dep. Cali', '0', '–', '2', 'Junior', '3.72', '3.16', '2.08', '13'],
 ['18:00',
  'Deportes Tolima',
  '0',
  '–',
  '1',
  'Aguilas',
 

In [78]:
rowData

[['20:00',
  'Ind. Medellin',
  '2',
  '–',
  '2',
  'Junior',
  'pen.',
  '1.72',
  '3.55',
  '4.95',
  '13'],
 ['16:00',
  'Junior',
  '3',
  '–',
  '2',
  'Ind. Medellin',
  '1.96',
  '3.19',
  '4.18',
  '13'],
 ['20:15',
  'Ind. Medellin',
  '2',
  '–',
  '1',
  'America De Cali',
  '2.25',
  '3.31',
  '3.17',
  '13'],
 ['20:15',
  'Junior',
  '4',
  '–',
  '2',
  'Deportes Tolima',
  '1.90',
  '3.64',
  '3.82',
  '13'],
 ['18:00',
  'Aguilas',
  '3',
  '–',
  '1',
  'Dep. Cali',
  '1.38',
  '4.70',
  '8.21',
  '13'],
 ['18:00',
  'Millonarios',
  '0',
  '–',
  '1',
  'Atl. Nacional',
  '1.83',
  '3.41',
  '4.45',
  '13'],
 ['19:00',
  'America De Cali',
  '1',
  '–',
  '0',
  'Millonarios',
  '2.39',
  '3.18',
  '3.05',
  '13'],
 ['15:30',
  'Atl. Nacional',
  '0',
  '–',
  '5',
  'Ind. Medellin',
  '2.93',
  '3.22',
  '2.45',
  '13'],
 ['20:15', 'Dep. Cali', '0', '–', '2', 'Junior', '3.72', '3.16', '2.08', '13'],
 ['18:00',
  'Deportes Tolima',
  '0',
  '–',
  '1',
  'Aguilas',
 

In [75]:
rowData

[['Football',
  '/',
  'Colombia',
  '/',
  'Primera A 2023',
  '13 Dec 2023 - Clausura - Play Offs',
  '1',
  'X',
  '2',
  "B's",
  '20:00',
  'Ind. Medellin',
  '2',
  '–',
  '2',
  'Junior',
  'pen.',
  '1.72',
  '3.55',
  '4.95',
  '13',
  '10 Dec 2023 - Clausura - Play Offs',
  '1',
  'X',
  '2',
  "B's",
  '16:00',
  'Junior',
  '3',
  '–',
  '2',
  'Ind. Medellin',
  '1.96',
  '3.19',
  '4.18',
  '13',
  '06 Dec 2023 - Clausura - Quadrangular',
  '1',
  'X',
  '2',
  "B's",
  '20:15',
  'Ind. Medellin',
  '2',
  '–',
  '1',
  'America De Cali',
  '2.25',
  '3.31',
  '3.17',
  '13',
  '20:15',
  'Junior',
  '4',
  '–',
  '2',
  'Deportes Tolima',
  '1.90',
  '3.64',
  '3.82',
  '13',
  '18:00',
  'Aguilas',
  '3',
  '–',
  '1',
  'Dep. Cali',
  '1.38',
  '4.70',
  '8.21',
  '13',
  '18:00',
  'Millonarios',
  '0',
  '–',
  '1',
  'Atl. Nacional',
  '1.83',
  '3.41',
  '4.45',
  '13',
  '03 Dec 2023 - Clausura - Quadrangular',
  '1',
  'X',
  '2',
  "B's",
  '19:00',
  'America D

In [76]:
matches = driver.find_elements(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/main/div[3]/div[4]/div[1]/div[1]/a')

[elem.get_attribute('href') for elem in matches]

[]

In [73]:
[row.text.split("\n") for row in gameRows]

[['Football',
  '/',
  'Colombia',
  '/',
  'Primera A 2023',
  '13 Dec 2023 - Clausura - Play Offs',
  '1',
  'X',
  '2',
  "B's",
  '20:00',
  'Ind. Medellin',
  '2',
  '–',
  '2',
  'Junior',
  'pen.',
  '1.72',
  '3.55',
  '4.95',
  '13',
  '10 Dec 2023 - Clausura - Play Offs',
  '1',
  'X',
  '2',
  "B's",
  '16:00',
  'Junior',
  '3',
  '–',
  '2',
  'Ind. Medellin',
  '1.96',
  '3.19',
  '4.18',
  '13',
  '06 Dec 2023 - Clausura - Quadrangular',
  '1',
  'X',
  '2',
  "B's",
  '20:15',
  'Ind. Medellin',
  '2',
  '–',
  '1',
  'America De Cali',
  '2.25',
  '3.31',
  '3.17',
  '13',
  '20:15',
  'Junior',
  '4',
  '–',
  '2',
  'Deportes Tolima',
  '1.90',
  '3.64',
  '3.82',
  '13',
  '18:00',
  'Aguilas',
  '3',
  '–',
  '1',
  'Dep. Cali',
  '1.38',
  '4.70',
  '8.21',
  '13',
  '18:00',
  'Millonarios',
  '0',
  '–',
  '1',
  'Atl. Nacional',
  '1.83',
  '3.41',
  '4.45',
  '13',
  '03 Dec 2023 - Clausura - Quadrangular',
  '1',
  'X',
  '2',
  "B's",
  '19:00',
  'America D

In [33]:
rowData

[['18:00',
  'Atl. Nacional',
  '2',
  '–',
  '0',
  'Junior',
  'pen.',
  '1.46',
  '4.11',
  '7.16',
  '6'],
 ['19:10',
  'Junior',
  '2',
  '–',
  '1',
  'Atl. Nacional',
  '2.39',
  '3.12',
  '3.00',
  '6'],
 ['19:30',
  'Junior',
  '1',
  '–',
  '0',
  'Deportes Tolima',
  '1.90',
  '3.38',
  '4.03',
  '6'],
 ['17:00',
  'Atl. Nacional',
  '2',
  '–',
  '0',
  'Ind. Medellin',
  '1.68',
  '3.57',
  '5.20',
  '6'],
 ['20:30',
  'Ind. Medellin',
  '1',
  '–',
  '0',
  'Atl. Nacional',
  '3.11',
  '3.07',
  '2.37',
  '6'],
 ['18:30',
  'Deportes Tolima',
  '0',
  '–',
  '1',
  'Junior',
  '1.78',
  '3.22',
  '4.42',
  '6'],
 ['19:30',
  'Ind. Medellin',
  '4',
  '–',
  '0',
  'Alianza',
  '1.52',
  '3.79',
  '6.78',
  '6'],
 ['17:00', 'Junior', '3', '–', '1', 'Santa Fe', '1.52', '3.80', '6.48', '6'],
 ['20:00',
  'Atl. Nacional',
  '3',
  '–',
  '1',
  'Dep. Cali',
  '1.44',
  '4.06',
  '7.46',
  '6'],
 ['17:30',
  'Deportes Tolima',
  '3',
  '–',
  '1',
  'Once Caldas',
  '1.69',
  

In [ ]:

columns = ['hora', 'h_team', 'h_score', 'marker', 'a_score', 'a_team', 'notes', 'OP_1', 'OP_X', 'OP_2', 'n_bookmkrs']

normalized_data = [
    row[:6] + [None] + row[6:] if len(row) == 10 else row  
    for row in rowData
]

df_event = pd.DataFrame(normalized_data, columns=columns)

df_event = df_event.fillna(value=np.nan)

df_event

,hora,h_team,h_score,marker,a_score,a_team,notes,OP_1,OP_X,OP_2,n_bookmkrs
0,18:00,Atl. Nacional,2,–,0,Junior,pen.,1.46,4.11,7.16,6
1,19:10,Junior,2,–,1,Atl. Nacional,NaN,2.39,3.12,3.00,6
2,19:30,Junior,1,–,0,Deportes Tolima,NaN,1.90,3.38,4.03,6
3,17:00,Atl. Nacional,2,–,0,Ind. Medellin,NaN,1.68,3.57,5.20,6
4,20:30,Ind. Medellin,1,–,0,Atl. Nacional,NaN,3.11,3.07,2.37,6
5,18:30,Deportes Tolima,0,–,1,Junior,NaN,1.78,3.22,4.42,6
6,19:30,Ind. Medellin,4,–,0,Alianza,NaN,1.52,3.79,6.78,6
7,17:00,Junior,3,–,1,Santa Fe,NaN,1.52,3.80,6.48,6
8,20:00,Atl. Nacional,3,–,1,Dep. Cali,NaN,1.44,4.06,7.46,6
9,17:30,Deportes Tolima,3,–,1,Once Caldas,NaN,1.69,3.43,5.17,6


In [40]:
# Define las columnas esperadas, incluyendo 'extra' antes de 'OP_1'
columns = ['hora', 'h_team', 'h_score', 'marker', 'a_score', 'a_team', 'extra', 'OP_1', 'OP_X', 'OP_2', 'n_bookmkrs']

# Normaliza cada fila: si tiene 10 elementos, agrega None después del sexto
normalized_data = [
    row[:6] + [None] + row[6:] if len(row) == 10 else row
    for row in rowData
]

# Crea el DataFrame usando solo las primeras columnas necesarias para el número máximo de columnas en la data
df_event = pd.DataFrame(normalized_data, columns=columns[:len(normalized_data[0])])

# Filtra filas con valores no deseados en 'extra', pero no elimina filas si 'extra' no existe o es válida
df_event = df_event[df_event['extra'].isna() | ~df_event['extra'].isin(['canc.', 'pen.', 'award.'])]

# Opcional: elimina la columna 'extra' si no es necesaria
#df_event.drop(columns='extra', inplace=True)

df_event

,hora,h_team,h_score,marker,a_score,a_team,extra,OP_1,OP_X,OP_2,n_bookmkrs
1,19:10,Junior,2,–,1,Atl. Nacional,None,2.39,3.12,3.00,6
2,19:30,Junior,1,–,0,Deportes Tolima,None,1.90,3.38,4.03,6
3,17:00,Atl. Nacional,2,–,0,Ind. Medellin,None,1.68,3.57,5.20,6
4,20:30,Ind. Medellin,1,–,0,Atl. Nacional,None,3.11,3.07,2.37,6
5,18:30,Deportes Tolima,0,–,1,Junior,None,1.78,3.22,4.42,6
6,19:30,Ind. Medellin,4,–,0,Alianza,None,1.52,3.79,6.78,6
7,17:00,Junior,3,–,1,Santa Fe,None,1.52,3.80,6.48,6
8,20:00,Atl. Nacional,3,–,1,Dep. Cali,None,1.44,4.06,7.46,6
9,17:30,Deportes Tolima,3,–,1,Once Caldas,None,1.69,3.43,5.17,6
10,19:30,Alianza,0,–,2,Ind. Medellin,None,2.65,3.00,2.77,6


In [27]:
rowData

[['18:00',
  'Atl. Nacional',
  '2',
  '–',
  '0',
  'Junior',
  'pen.',
  '1.46',
  '4.11',
  '7.16',
  '6'],
 ['19:10',
  'Junior',
  '2',
  '–',
  '1',
  'Atl. Nacional',
  '2.39',
  '3.12',
  '3.00',
  '6'],
 ['19:30',
  'Junior',
  '1',
  '–',
  '0',
  'Deportes Tolima',
  '1.90',
  '3.38',
  '4.03',
  '6'],
 ['17:00',
  'Atl. Nacional',
  '2',
  '–',
  '0',
  'Ind. Medellin',
  '1.68',
  '3.57',
  '5.20',
  '6'],
 ['20:30',
  'Ind. Medellin',
  '1',
  '–',
  '0',
  'Atl. Nacional',
  '3.11',
  '3.07',
  '2.37',
  '6'],
 ['18:30',
  'Deportes Tolima',
  '0',
  '–',
  '1',
  'Junior',
  '1.78',
  '3.22',
  '4.42',
  '6'],
 ['19:30',
  'Ind. Medellin',
  '4',
  '–',
  '0',
  'Alianza',
  '1.52',
  '3.79',
  '6.78',
  '6'],
 ['17:00', 'Junior', '3', '–', '1', 'Santa Fe', '1.52', '3.80', '6.48', '6'],
 ['20:00',
  'Atl. Nacional',
  '3',
  '–',
  '1',
  'Dep. Cali',
  '1.44',
  '4.06',
  '7.46',
  '6'],
 ['17:30',
  'Deportes Tolima',
  '3',
  '–',
  '1',
  'Once Caldas',
  '1.69',
  

In [164]:
pages_season = get_pages_season(link="https://www.oddsportal.com/football/colombia/liga-postobon-2014/results/")

In [165]:
pages_season

['1', '2', '3', '4', '5', '6', '7', '8']

In [184]:
list_results = []

for x in pages_season:

    df_current = get_season_data(link="https://www.oddsportal.com/football/colombia/liga-postobon-2014/results/",page=x)

    sleep(20)
    
    list_results.append(df_current)




In [185]:

df_consolidado = pd.concat(list_results)

In [186]:
df_consolidado

,hora,h_team,h_score,marker,a_score,a_team,OP_1,OP_X,OP_2,n_bookmkrs,...,William Hill_Payout,William Hill_X,bet-at-home_1,bet-at-home_2,bet-at-home_Payout,bet-at-home_X,bet365_1,bet365_2,bet365_Payout,bet365_X
0,17:30,Santa Fe,1,–,1,Ind. Medellin,1.85,3.51,4.03,6,...,91.4%,3.30,1.80,3.73,89.2%,3.36,1.85,4.20,94.0%,3.50
1,19:45,Uniautonoma,2,–,0,Quindio,1.75,3.32,4.94,4,...,91.9%,3.10,1.70,4.34,89.3%,3.32,1.75,5.00,93.8%,3.40
2,19:30,Ind. Medellin,1,–,2,Santa Fe,1.96,3.33,3.90,6,...,91.8%,3.10,1.94,3.52,89.4%,3.13,2.00,4.00,94.1%,3.20
3,15:30,Quindio,0,–,0,Uniautonoma,2.11,3.24,3.37,5,...,91.5%,3.00,2.07,3.15,89.4%,3.14,2.15,3.25,93.7%,3.40
4,18:15,Atl. Nacional,0,–,1,Santa Fe,1.52,3.83,6.46,6,...,91.1%,4.00,1.51,5.45,89.4%,3.66,1.53,7.00,94.1%,3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,17:30,Once Caldas,3,–,1,Fortaleza,1.57,3.76,5.43,5,...,90.0%,3.50,1.50,5.00,88.5%,3.80,1.57,6.00,93.4%,3.75
12,17:00,Envigado,1,–,2,Millonarios,2.99,3.17,2.28,5,...,92.0%,3.10,2.90,2.20,89.1%,3.10,3.00,2.20,90.1%,3.10
13,15:15,Uniautonoma,3,–,2,Deportes Tolima,2.71,3.18,2.46,5,...,-,3.10,2.60,2.40,89.0%,3.10,2.45,2.63,90.0%,3.10
14,20:00,Santa Fe,3,–,0,Aguilas,1.53,3.91,5.62,5,...,91.0%,4.00,1.50,5.05,88.9%,3.85,1.57,5.25,91.4%,3.75


In [159]:
driver.get("https://www.oddsportal.com/football/colombia/liga-postobon-2014/results/")

driver.refresh()

sleep(5)

pages = driver.find_elements(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/main/div[3]/div[4]/div[1]/div[3]/div/a')

pages_list = [elem.text for elem in pages]

filtered_list = [
    item for item in pages_list
    if item != 'Next' 
]

print(filtered_list)

['1', '2', '3', '4', '5', '6', '7', '8']


In [170]:
page = filtered_list[0]
link = 'https://www.oddsportal.com/football/colombia/liga-postobon-2014/results/#/page/{}'.format(page)
print(link)


https://www.oddsportal.com/football/colombia/liga-postobon-2014/results/#/page/1


In [ ]:
driver.get(link)

driver.refresh()

sleep(5)

gameRows = driver.find_elements(By.XPATH,'//div[@class="group flex"]')

rowData = [row.text.split("\n") for row in gameRows]

df_event = pd.DataFrame(rowData, columns=['hora', 'h_team', 'h_score', 'marker', 'a_score', 'a_team', 'OP_1', 'OP_X', 'OP_2', 'n_bookmkrs'])

print(df_event)

In [ ]:
matches = driver.find_elements(By.XPATH,'//div[@class="group flex"]/a')

matches_links = [elem.get_attribute('href') for elem in matches]

m_link = matches_links[0]

print(m_link)

In [ ]:

driver.get(m_link)

bookmarkers = driver.find_elements(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/main/div[3]/div[2]/div[2]/div/div/div[1]')

bookmarkers_table = [row.text.split("\n") for row in bookmarkers]

filtered_data = [
    item for item in bookmarkers_table[0] 
    if item != 'BONUS' and not item.startswith('CLICK TO SHOW')
]

# Dividir la lista en grupos de 5 elementos
rows = [filtered_data[i:i + 5] for i in range(0, len(filtered_data), 5)]

# Crear el DataFrame
df = pd.DataFrame(rows[1:], columns=rows[0])  # Ignoramos la primera fila ya que es el header

df_2 = pd.melt(df, id_vars='Bookmakers', value_vars=['1', 'X', '2','Payout'])

df_2["Var"] = df_2['Bookmakers'] +"_"+ df_2["variable"]

result = df_2[["Var", "value"]].transpose()

result.columns = result.iloc[0]
result = result[1:]

result.reset_index(drop=True, inplace=True)

result = result.reindex(sorted(result.columns), axis=1)

# Print the result
print(result)

# BORRADOR

In [ ]:
rowData[0][0]
rowData[0][1]
rowData[0][2]
rowData[0][3]
rowData[0][4]
rowData[0][5]
rowData[0][6]
rowData[0][7]
rowData[0][8]
rowData[0][9]

In [50]:
gameRows = driver.find_elements(By.XPATH,'//div[@class="group flex"]')

In [37]:
data = {}

In [56]:
len(gameRows)

0

In [51]:
rowData = [row.text.split("\n") for row in gameRows]

In [52]:
print(len(gameRows))

0


In [ ]:
for row in rowData:
    if row[2] == "\u2013":
        try:
            data.update({row[1]+" vs. "+row[3]:{
                                        "Team1_Odds":row[4],
                                        "Team2_Odds":row[5],
                                        "No. of Bookmakers":row[6]
                                        }})
        except:
            continue
    else:
        try:
            data.update({row[1]+" vs. "+row[5]:{
                                        "Team1_Odds":row[6],
                                        "Team2_Odds":row[7],
                                        "No. of Bookmakers":row[8]
                                        }})
        except:
            continue


with open("NBA.json","w") as jsonFile:
    json.dump(data,jsonFile,indent=4)

In [53]:
rowData

[]

In [54]:
rowData[0][0]

IndexError: list index out of range